In [2]:
!pip install pyswarm


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyswarm: filename=pyswarm-0.6-py3-none-any.whl size=4486 sha256=1eff7c1527bf767176e691a536ff5f505ed692aa8cd716319969004a93afa986
  Stored in directory: c:\users\12257883\appdata\local\pip\cache\wheels\93\15\89\3970ef96abd6123028010a90f007c4e6a2bed700db0aa2d36a
Successfully built pyswarm


In [3]:
# Import Required Libraries

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from pyswarm import pso  # Particle Swarm Optimization


In [4]:
# Load and Preprocess MNIST Dataset

# Load dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
# Normalize and flatten images
x_train = x_train.reshape(-1, 28*28) / 255.0
x_test = x_test.reshape(-1, 28*28) / 255.0

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Optional: Use a subset to speed up training
x_train, _, y_train, _ = train_test_split(x_train, y_train, train_size=0.2, random_state=42)

In [6]:
# Define Model Builder with Custom Hyperparameters

def build_model(hparams):
    learning_rate = hparams[0]
    neurons = int(hparams[1])
    dropout_rate = hparams[2]
    
    model = Sequential([
        Dense(neurons, activation='relu', input_shape=(784,)),
        Dropout(dropout_rate),
        Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [7]:
# Objective Function for PSO

def fitness(hparams):
    learning_rate, neurons, dropout = hparams
    if not (0.0001 <= learning_rate <= 0.01 and 32 <= neurons <= 128 and 0.1 <= dropout <= 0.5):
        return 1.0  # Penalize invalid ranges
    
    model = build_model(hparams)
    history = model.fit(x_train, y_train, epochs=3, batch_size=64, verbose=0)
    loss, acc = model.evaluate(x_test, y_test, verbose=0)
    
    return 1 - acc  # Minimize (1 - accuracy)


In [8]:
# Run PSO to Find Optimal Hyperparameters

# Define bounds: [learning_rate, neurons, dropout]
lb = [0.0001, 32, 0.1]
ub = [0.01, 128, 0.5]

best_params, best_score = pso(fitness, lb, ub, swarmsize=10, maxiter=5)
print("Best Hyperparameters (PSO):", best_params)


C:\Users\12257883\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Stopping search: maximum iterations reached --> 5
Best Hyperparameters (PSO): [7.72808886e-03 1.21795997e+02 3.39108141e-01]


In [9]:
# Traditional Method – Grid Search

from itertools import product

# Define grid values
lr_values = [0.001, 0.005]
neuron_values = [64, 128]
dropout_values = [0.2, 0.3]

best_acc = 0
best_combo = None

for lr, n, d in product(lr_values, neuron_values, dropout_values):
    model = build_model([lr, n, d])
    model.fit(x_train, y_train, epochs=3, batch_size=64, verbose=0)
    _, acc = model.evaluate(x_test, y_test, verbose=0)
    
    if acc > best_acc:
        best_acc = acc
        best_combo = (lr, n, d)

print("Best Hyperparameters (Grid):", best_combo)


Best Hyperparameters (Grid): (0.005, 128, 0.3)


In [10]:
# Final Comparison

print("PSO Best Accuracy: {:.2f}%".format((1 - best_score) * 100))
print("Grid Search Best Accuracy: {:.2f}%".format(best_acc * 100))


PSO Best Accuracy: 95.32%
Grid Search Best Accuracy: 95.38%
